In [1]:
!df -h

Filesystem      Size  Used Avail Use% Mounted on
overlay          48G   24G   22G  53% /
tmpfs            64M     0   64M   0% /dev
tmpfs            45G     0   45G   0% /sys/fs/cgroup
shm             1.0G     0  1.0G   0% /dev/shm
/dev/xvda1       48G   24G   22G  53% /etc/hosts
tmpfs            45G   12K   45G   1% /proc/driver/nvidia
udev             45G     0   45G   0% /dev/nvidia0
tmpfs            45G     0   45G   0% /proc/acpi
tmpfs            45G     0   45G   0% /proc/scsi
tmpfs            45G     0   45G   0% /sys/firmware


## Library

In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())

1.7.1
True


In [3]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

/opt/conda/lib/python3.7/site-packages/mmcv/utils/registry.py:72: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '
/opt/conda/lib/python3.7/site-packages/mmcv/utils/registry.py:72: UserWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  'The old API of register_module(module, force=False) '


## Load Config

In [4]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

model_name = 'vfnet_r2_101_fpn_mdconv_c3-c5_mstrain_2x_coco'
folder_name = 'vfnet'

# config file 들고오기
cfg = Config.fromfile(f'./configs/{folder_name}/{model_name}.py')

## Hyper-parameters

In [5]:
# batch size
cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = 4

# epochs
# cfg.runner.max_epochs = 25 

# learning rate
# cfg.optimizer.lr = 1e-4

# scheduler
# cfg.lr_config = dict(
#     policy='CosineAnnealing',
#     warmup='linear',
#     warmup_iters=1000,
#     warmup_ratio=1.0 / 10,
#     min_lr_ratio=1e-6
# )

## Config Check

In [6]:
import pprint
pprint.pprint(cfg.lr_config)

{'policy': 'step',
 'step': [16, 22],
 'warmup': 'linear',
 'warmup_iters': 500,
 'warmup_ratio': 0.1}


In [7]:
pprint.pprint(cfg.optimizer)

{'lr': 0.01,
 'momentum': 0.9,
 'paramwise_cfg': {'bias_decay_mult': 0.0, 'bias_lr_mult': 2.0},
 'type': 'SGD',
 'weight_decay': 0.0001}


In [8]:
pprint.pprint(cfg.data)

{'samples_per_gpu': 8,
 'test': {'ann_file': 'data/coco/annotations/instances_val2017.json',
          'img_prefix': 'data/coco/val2017/',
          'pipeline': [{'type': 'LoadImageFromFile'},
                       {'flip': False,
                        'img_scale': (1333, 800),
                        'transforms': [{'keep_ratio': True, 'type': 'Resize'},
                                       {'type': 'RandomFlip'},
                                       {'mean': [123.675, 116.28, 103.53],
                                        'std': [58.395, 57.12, 57.375],
                                        'to_rgb': True,
                                        'type': 'Normalize'},
                                       {'size_divisor': 32, 'type': 'Pad'},
                                       {'type': 'DefaultFormatBundle'},
                                       {'keys': ['img'], 'type': 'Collect'}],
                        'type': 'MultiScaleFlipAug'}],
          'type': 'CocoDataset

In [9]:
pprint.pprint(cfg.model)

{'backbone': {'base_width': 26,
              'dcn': {'deform_groups': 1,
                      'fallback_on_stride': False,
                      'type': 'DCNv2'},
              'depth': 101,
              'frozen_stages': 1,
              'norm_cfg': {'requires_grad': True, 'type': 'BN'},
              'norm_eval': True,
              'num_stages': 4,
              'out_indices': (0, 1, 2, 3),
              'scales': 4,
              'stage_with_dcn': (False, True, True, True),
              'style': 'pytorch',
              'type': 'Res2Net'},
 'bbox_head': {'center_sampling': False,
               'dcn_on_last_conv': True,
               'feat_channels': 256,
               'in_channels': 256,
               'loss_bbox': {'loss_weight': 1.5, 'type': 'GIoULoss'},
               'loss_bbox_refine': {'loss_weight': 2.0, 'type': 'GIoULoss'},
               'loss_cls': {'alpha': 0.75,
                            'gamma': 2.0,
                            'iou_weighted': True,
           

## Config Tuning

In [10]:
PREFIX = '../input/data/'


# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
# cfg.data.train.pipeline[2]['img_scale'] = (512, 512)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
# cfg.data.val.pipeline[1]['img_scale'] = (512, 512)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
# cfg.data.test.pipeline[1]['img_scale'] = (512, 512)


cfg.seed=0
cfg.gpu_ids = [0]
cfg.work_dir = f'./work_dirs/{model_name}'

cfg.model.bbox_head.num_classes = 11

cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)

## Train

In [11]:
model = build_detector(cfg.model)

2021-05-19 09:20:40,446 - mmdet - INFO - load model from: open-mmlab://res2net101_v1d_26w_4s
2021-05-19 09:20:40,446 - mmdet - INFO - Use load_from_openmmlab loader
2021-05-19 09:20:40,795 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.convs.0.conv_offset.weight, layer2.0.convs.0.conv_offset.bias, layer2.0.convs.1.conv_offset.weight, layer2.0.convs.1.conv_offset.bias, layer2.0.convs.2.conv_offset.weight, layer2.0.convs.2.conv_offset.bias, layer2.1.convs.0.conv_offset.weight, layer2.1.convs.0.conv_offset.bias, layer2.1.convs.1.conv_offset.weight, layer2.1.convs.1.conv_offset.bias, layer2.1.convs.2.conv_offset.weight, layer2.1.convs.2.conv_offset.bias, layer2.2.convs.0.conv_offset.weight, layer2.2.convs.0.conv_offset.bias, layer2.2.convs.1.conv_offset.weight, layer2.2.convs.1.conv_offset.bias, layer2.2.convs.2.conv_offset.weight, layer2.2.convs.2.conv_offset.bia

In [12]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.17s)
creating index...
index created!


In [13]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-19 09:20:49,916 - mmdet - INFO - load checkpoint from ./pretrained/vfnet_r2_101_dcn_ms_2x_51.1.pth
2021-05-19 09:20:49,917 - mmdet - INFO - Use load_from_local loader


Done (t=0.79s)
creating index...
index created!


2021-05-19 09:20:50,393 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.vfnet_cls.weight: copying a param with shape torch.Size([80, 256, 3, 3]) from checkpoint, the shape in current model is torch.Size([11, 256, 3, 3]).
size mismatch for bbox_head.vfnet_cls.bias: copying a param with shape torch.Size([80]) from checkpoint, the shape in current model is torch.Size([11]).
2021-05-19 09:20:50,407 - mmdet - INFO - Start running, host: root@f10ce2c11230, work_dir: /opt/ml/VarifocalNet/work_dirs/vfnet_r2_101_fpn_mdconv_c3-c5_mstrain_2x_coco
2021-05-19 09:20:50,407 - mmdet - INFO - workflow: [('train', 1)], max: 24 epochs
/opt/ml/VarifocalNet/mmdet/core/evaluation/eval_hooks.py:112: UserWarning: runner.meta is None. Creating a empty one.
  warnings.warn('runner.meta is None. Creating a empty one.')
2021-05-19 09:22:55,144 - mmdet - INFO - Epoch [1][50/328]	lr: 1.882e-03, eta: 5:25:12, time: 2.495, data_time: 0.100, memory: 22489, loss_cls

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 7.7 task/s, elapsed: 85s, ETA:     0s

...

2021-05-19 14:00:21,039 - mmdet - INFO - Epoch(val) [19][328]	bbox_mAP: 0.3640, bbox_mAP_50: 0.4810, bbox_mAP_75: 0.3830, bbox_mAP_s: 0.1040, bbox_mAP_m: 0.3450, bbox_mAP_l: 0.5930, bbox_mAP_copypaste: 0.364 0.481 0.383 0.104 0.345 0.593


DONE (t=1.55s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.364
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.481
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.383
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.104
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.345
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.593
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.565
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.565
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.565
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.264
Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.560
Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.824


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f5c42de6a70>
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1203, in __del__
    self._shutdown_workers()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 1177, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 140, in join
    res = self._popen.wait(timeout)
  File "/opt/conda/lib/python3.7/multiprocessing/popen_fork.py", line 45, in wait
    if not wait([self.sentinel], timeout):
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 920, in wait
    ready = selector.select(timeout)
  File "/opt/conda/lib/python3.7/selectors.py", line 415, in select
    fd_event_list = self._selector.poll(timeout)
KeyboardInterrupt: 
ERROR:root:Internal Python error in the inspect module.
Below is the traceback fro

Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-13-59b5768e9e45>", line 1, in <module>
    train_detector(model, datasets[0], cfg, distributed=False, validate=True)
  File "/opt/ml/VarifocalNet/mmdet/apis/train.py", line 175, in train_detector
    runner.run(data_loaders, cfg.workflow)
  File "/opt/conda/lib/python3.7/site-packages/mmcv/runner/epoch_based_runner.py", line 125, in run
    epoch_runner(data_loaders[i], **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/mmcv/runner/epoch_based_runner.py", line 50, in train
    self.run_iter(data_batch, train_mode=True)
  File "/opt/conda/lib/python3.7/site-packages/mmcv/runner/epoch_based_runner.py", line 30, in run_iter
    **kwargs)
  File "/opt/conda/lib/python3.7/site-packages/mmcv/parallel/data_parallel.py", line 67, in train_step
    return self.module.train_st


KeyboardInterrupt



## Save Config

In [ ]:
import os
from mmcv import Config

# ./configs/{model_name}/{model_cfg}
# Example)
# ./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py

# config 불러오기.
cfg = Config.fromfile(os.path.join("./configs", folder_name, f'{model_name}.py'))

# config 출력하기.
# 모든 설정 값이 출력됩니다요.
print(cfg.dump())

# 현재 경로의 _custom_configs_ 폴더를 만들고 거기다가 config 파일을 저장합니다.
os.makedirs(os.path.join("_custom_configs_"), exist_ok=True)
cfg.dump(os.path.join("_custom_configs_", model_name))